In [6]:
import numpy as np
import matplotlib.pyplot as plt
import torch

import sys
sys.path.append("..")

from data import utils
from data import load_raw_data
from models.datasets import SorghumDatasetWithNormals

In [2]:
pcd = load_raw_data.load_pcd_plyfile_new_approach("/space/ariyanzarei/sorghum_segmentation/dataset/archive/synthetic/2022-11-06/PointCloud/S11_31.ply", True)
points = pcd['points']
labels = pcd['labels'].squeeze()

In [ ]:
maxs = np.max(points,0)

ind = np.where(points[:,2] == maxs[2])[0][0]

points = torch.from_numpy(points).type(torch.DoubleTensor).unsqueeze(0)

mins, _ = torch.min(points, axis=1)
maxs, _ = torch.max(points, axis=1)
mins = mins.unsqueeze(1)
maxs = maxs.unsqueeze(1)
points = (points - mins) / (maxs - mins) - 0.5

points = points.squeeze()
distance = torch.cdist(points, points)

knn_values, knn_ind = (distance * -1).topk(k=20, dim=-1)
nonzero_ind = (knn_values<0).nonzero()
print(knn_values.shape, nonzero_ind.shape)
# filtered_ind = knn_ind[]
print(filtered_ind.shape)
knn_pred_dist = distance.gather(-1, knn_ind)

print(knn_ind.shape, knn_pred_dist.shape)
print(min(distance[ind]),max(distance[ind]))

close_points = np.where(distance[ind]<0.1)
far_points = np.where(distance[ind]>=0.1)
distance_colors = distance[ind]

d_colors = utils.distinct_colors(len(set(labels.tolist())))
colors = np.zeros((labels.shape[0], 3))
for i, l in enumerate(list(set(labels.tolist()))):
    colors[labels == l, :] = d_colors[i]

fig = plt.figure(figsize=(15,15))
ax = fig.add_subplot(projection='3d')
ax.scatter(points[far_points,0],points[far_points,1],points[far_points,2], s=2)
ax.scatter(points[ind,0],points[ind,1],points[ind,2], s=20, c=(1,0,0))
ax.scatter(points[close_points,0],points[close_points,1],points[close_points,2], s=5, c=(0,1,0))
# ax.scatter(points[:,2],points[:,0],points[:,1], s=5, c=distance_colors)
plt.show()


In [7]:
ds = SorghumDatasetWithNormals("/space/ariyanzarei/sorghum_segmentation/dataset/SPNS/SorghumPartNetFormat/instance_segmentation_train.hdf5", False)

In [9]:
ind = 3

points, labels = ds[ind:ind+2]
print(points.shape)
print(labels.shape)

torch.Size([2, 8000, 3])
torch.Size([2, 8000])


In [19]:
def knn(x, k):
    inner = -2 * torch.matmul(x.transpose(2, 1), x)
    xx = torch.sum(x**2, dim=1, keepdim=True)
    pairwise_distance = -xx - inner - xx.transpose(2, 1)
    idx = pairwise_distance.topk(k=k, dim=-1)[1]  # (batch_size, num_points, k)
    return idx

In [30]:
k = 5

x = points.transpose(1,2)
batch_size = x.size(0)
num_points = x.size(2)
x = x.view(batch_size, -1, num_points)
idx = knn(x, k)

idx_base = torch.arange(0, batch_size).view(-1, 1, 1) * num_points
idx = idx + idx_base
idx = idx.view(-1)
_, num_dims, _ = x.size()
x = x.transpose(2, 1).contiguous()
knn_coords = x.view(batch_size * num_points, -1)[idx, :]
knn_coords = knn_coords.view(batch_size, num_points, k, num_dims)
print(knn_coords.shape)

torch.Size([2, 8000, 5, 3])
